Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_s("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
df2[:height] = convert(Vector{Float64}, df2[:height]);
df2[:weight] = convert(Vector{Float64}, df2[:weight]);
df2[:weight_s] = (df2[:weight] .- mean(df2[:weight])) / std(df2[:weight]);
df2[:weight_s2] = df2[:weight_s] .^ 2;

Define the Stan language model

In [5]:
weightsmodel = "
data{
    int N;
    real height[N];
    real weight_s2[N];
    real weight_s[N];
}
parameters{
    real a;
    real b1;
    real b2;
    real sigma;
}
model{
    vector[N] mu;
    sigma ~ uniform( 0 , 50 );
    b2 ~ normal( 0 , 10 );
    b1 ~ normal( 0 , 10 );
    a ~ normal( 178 , 100 );
    for ( i in 1:N ) {
        mu[i] = a + b1 * weight_s[i] + b2 * weight_s2[i];
    }
    height ~ normal( mu , sigma );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["a", "b1", "b2", "sigma"],
model=weightsmodel,  output_format=:mcmcchains);


File /Users/rob/.julia/dev/StanModels/scripts/04/tmp/weights.stan will be updated.



Input data for cmdstan

In [7]:
heightsdata = Dict("N" => size(df2, 1), "height" => df2[:height],
"weight_s" => df2[:weight_s], "weight_s2" => df2[:weight_s2]);

Sample using cmdstan

In [8]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -3629.51, but must be > 0!  (in '/Users/rob/.julia/dev/StanModels/scripts/04/tmp/weights.stan' at line 22)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -3602.34, but must be > 0!  (in '/Users/rob/.julia/dev/StanModels/scripts/04/tmp/weights.stan' at line 22)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecif

Describe the draws

In [9]:
describe(chn)

2-element Array{ChainDataFrame,1}

Summary Statistics

│ Row │ parameters │ mean       │ std      │ naive_se   │ mcse       │ ess    │
│     │ Symbol     │ Float64    │ Float64  │ Float64    │ Float64    │ Any    │
├─────┼────────────┼────────────┼──────────┼────────────┼────────────┼────────┤
│ 1   │ a          │ 154.608    │ 0.357846 │ 0.00565804 │ 0.00793399 │ 4000.0 │
│ 2   │ b1         │ 5.84111    │ 0.265523 │ 0.00419829 │ 0.00514735 │ 4000.0 │
│ 3   │ b2         │ -0.0122503 │ 0.224123 │ 0.0035437  │ 0.00487386 │ 4000.0 │
│ 4   │ sigma      │ 5.11089    │ 0.193041 │ 0.00305225 │ 0.00257227 │ 4000.0 │

Quantiles

│ Row │ parameters │ 2.5%      │ 25.0%     │ 50.0%      │ 75.0%    │ 97.5%    │
│     │ Symbol     │ Float64   │ Float64   │ Float64    │ Float64  │ Float64  │
├─────┼────────────┼───────────┼───────────┼────────────┼──────────┼──────────┤
│ 1   │ a          │ 153.887   │ 154.37    │ 154.609    │ 154.847  │ 155.293  │
│ 2   │ b1         │ 5.30749   │ 5.66469   │ 5.83699 

End of `m4.5s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*